In [ ]:
import os
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git

%cd {HOME}/GroundingDINO
!pip install -e .
!pip install -q roboflow

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))


%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

In [ ]:
import os
import supervision as sv
import cv2
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

# label_folder = "label_folder_path"
# vid_folder = "vid_folder_path"

# target = "vid_file_name"


TEXT_PROMPT = "pigeon"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25


vid_image_folder = os.path.join(vid_folder, target)
vid_label_folder = os.path.join(labels_folder,target)

for IMAGE_PATH in tqdm(glob(vid_image_folder + "/*.png")):

  image_source, image = load_image(IMAGE_PATH)

  boxes, logits, phrases = predict(
      model=model,
      image=image,
      caption=TEXT_PROMPT,
      box_threshold=BOX_TRESHOLD,
      text_threshold=TEXT_TRESHOLD
  )

  text_file_path = os.path.join(vid_label_folder, os.path.basename(IMAGE_PATH)[:-4] + ".txt")
  boxes_list = boxes.tolist()

  with open(text_file_path, "w") as f:
    for box in boxes_list:
        line = "0"
        for item in box:
            line += (" " + str(item))
        line += "\n"

        f.write(line)